# 知识工程-问答系统设计与实现大作业
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本文档理论上支持多个格式，包括ipynb, docx, pdf 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对原本代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 数据准备

非常好的助教已经帮我们down好了数据，注意到有四个文件
- passages_multi_sentences.json (文档库) 
- train.json (问答语料) 
- train_questions.txt (问题分类训练语料) 
- test_questions.txt (问题分类测试语料) 

我们自己查看一下数据格式

![alt text](image.png)

![alt text](image-1.png)


| 文件名 (File Name)             | 来源数据集 (Source Dataset)                                  | 数据集描述 (Dataset Description)                                                                                                                               | 文件格式 (File Format)                                                                                                                                  |
| :----------------------------- | :----------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `passages_multi_sentences.json` | 百度 DuReader V2.0 (筛选处理后)                     | 包含用于检索和答案抽取的文档库，这些文档被分割成了多个句子 。                                                                                             | 实际上是JSONL，每一行是JSON 格式。每一个json有pid字段和document 字段，document是句子字符串的列表，每个字符串逻辑上是一个Sentence。                                                                                                                       |
| `train.json`                   | 百度 DuReader V2.0 (筛选处理后)                     | 问答语料，包含问题、对应的答案以及相关的文档 `pid` 。用于训练检索评估、候选答案句排序和答案抽取模块。                                                    | 也是JSONL，每一行的对象是question、pid、          answer_sentence、   answer、qid组成的，其中answer_sentence是句子的列表，但是一般是一个句子，answer和question是单个字符串，pid和qid是int                                                          |
| `train_questions.txt`          | 哈工大信息检索研究室问答系统问题集                      | 用于训练问题分类器的问题及对应的类别标签 。类别标签形如 `HUM_PERSON`，下划线前为粗分类标签 。                                                   | 文本文件 (.txt)。每行可能是一个标签和一个问题，由制表符或空格分隔。标签看起来是这个问题的类型，比如DES_DEFINITION是要求别人描述定义的问题。                                                                                                       |
| `test_questions.txt`           | 哈工大信息检索研究室问答系统问题集                      | 用于测试问题分类器性能的问题及对应的类别标签 。                                                                    | 与`train_questions.txt`一样。                                                                                                     |


主要来自两个数据集 ：

- DuReader V2.0：这是百度在2017年发布的一个阅读理解数据集 。本次实验使用的 passages_multi_sentences.json 和 train.json 就是从这个数据集中筛选和处理得到的 。
- 哈工大信息检索研究室问答系统问题集：这是一个由哈工大信息检索研究室在2004年公开的数据集 。实验中的问题分类语料（即 train_questions.txt 和 test_questions.txt）来源于此 。

## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image-2.png)


很好，这次代码格式化没有报错。

Pylance 似乎也没有明显问题。

## 实验环境准备

采用上次的作业专属环境，为了跑通最新方法，使用3.12 和 torch 2.7

```bash
conda create -n ai python=3.12
conda activate ai
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

本次作业似乎没有新的依赖，只是用到了 torch


## 原理回顾和课件复习



在作业9. 中我们进行了简单的复习，这里不再赘述。

## 1. 建立文档检索系统

首先我们来理解这个子任务的要求，
我注意到，train.json中的样本的pid，其实就是passages中的pid，是一一对应的。
我们要构建一个函数，输入train.json中的question，中间经过关键词提取, 系统读取passages_multi_sentences.json中的文档库，返回与问题相关的文档的pid列表， 使用Recall指标来评估检索系统的性能。

这里没有让我们用深度学习，而是传统的检索。

首先需要去除停用词，停用词是指在文本中出现频率非常高，但对文本意义贡献不大的词语（如“的”、“是”、“在”等）。去除它们可以减少噪音，降低计算量。注意到助教已经提供了停用词列表 `stopwords.txt`，我们可以直接使用。



In [ ]:
def load_stopwords(filepath):
    """Loads stopwords from a file into a set."""
    stopwords_set = set()
    if not os.path.exists(filepath):
        print(f"Warning: Stopwords file not found at {filepath}. Proceeding without stopwords.")
        return stopwords_set
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            stopwords_set.add(line.strip())
    print(f"Loaded {len(stopwords_set)} stopwords.")
    return stopwords_set

在助教的作业要求中，建议我们使用Whoosh https://whoosh.readthedocs.io/en/latest/intro.html

Whoosh，像其他搜索引擎库一样，通常在构建索引时内部处理文本分析（分词、去除停用词、小写化、词干提取等）。这是通过其 Schema 中的 FieldType（尤其是 TEXT 类型字段）及其关联的 Analyzer 来完成的。

作业要求中助教让我们实现的 Jieb分词和停用词去除逻辑，实际上正是 Whoosh Analyzer 需要做的事情。所以更理想和“Whoosh原生”的做法是，我们不需要显示地写一个 preprocess_data.py 把输出（词列表）直接保存成一个中间文件让 Whoosh 去读取。而是，我们将原始文本（passages_multi_sentences.json 中的句子）直接喂给 Whoosh，然后配置 Whoosh 使用一个包含 Jieba 分词和我们停用词表的自定义 Analyzer。这样，索引的建立和文本分析的逻辑就都封装在 Whoosh 内部了，更加清晰和高效。

查阅 https://whoosh.readthedocs.io/en/latest/analysis.html

接口需要我们对str进行yield Token对象。

In [ ]:
# --- Step a) Part 1: Preprocessing logic for Whoosh Analyzer and Questions ---
class ChineseTokenizer(Tokenizer):
    """Custom Whoosh Tokenizer for Chinese text using Jieba."""
    def __call__(self, text:str, **kargs):
        words = jieba.lcut(text) 
        token_instance = Token() # Create one Token object to reuse
        current_pos = 0
        for word in words:
            word_clean = word.strip()
            # Ensure it's not a stopword and not just whitespace
            if word_clean and word_clean.lower() not in STOPWORDS and not word_clean.isspace():
                token_instance.text = word_clean
                token_instance.original = word_clean
                token_instance.pos = current_pos # Assign current position
                # token_instance.startchar = start_char # Optional: if you track character offsets
                # token_instance.endchar = end_char   # Optional
                yield token_instance
                current_pos += 1

Whoosh Schema 我们需要定义一个 Schema 来描述文档的结构。至少会有一个字段用于存储文档ID (pid)，以及一个 TEXT 字段用于存储文档内容，这个内容字段将使用我们自定义的 Analyzer。

In [ ]:
schema = Schema(
        pid=ID(stored=True, unique=True),
        content=TEXT(analyzer=chinese_analyzer(), stored=False) # Content not stored to save space
    )

现在对于每条记录，提取 pid 和 document (句子列表)。将句子列表合并为单个字符串。
使用 IndexWriter.add_document() 将 pid 和合并后的文档内容添加到索引中。

In [ ]:
full_document_text = " ".join(document_sentences)

                    writer.add_document(pid=str(pid), content=full_document_text)

writer.commit()

我们只写了关键核心代码，其他读取json等细节问题就没写了。关键是知道Whoosh就是用writer来写入索引，检索的时候不区分文档的句子，只是检索文档。

然后是推理的时候，给定一个问题，要怎么查询pid。

In [ ]:
def search_documents(ix, question_text, top_n=3):
    """
    Searches the index for a given question and returns top_n pids.
    (Addresses part of step c)
    """
    # Preprocess the question to get keywords (also part of step c)
    query_keywords = preprocess_question(question_text)
    if not query_keywords:
        return []

    # Form a query string (Whoosh OR query by default for space-separated terms)
    # Using OR logic, as suggested by "命中一个关键词加一" then sum scores.
    # Whoosh's default OR for space-separated terms is a good starting point.
    query_str = " ".join(query_keywords)
    
    retrieved_pids = []
    with ix.searcher() as searcher:
        # Using MultifieldParser to search in the 'content' field.
        # You could use QueryParser for a single field too.
        # Default operator is OR. To make it AND, use qparser.AND
        query_parser = QueryParser("content", schema=ix.schema)
        query = query_parser.parse(query_str)
        
        results = searcher.search(query, limit=top_n)
        for hit in results:
            retrieved_pids.append(hit['pid']) # pid is stored as string in index
            
    return retrieved_pids

这一步关键是要用 QueryParser 和 searcher

现在我们运行

python src/retrieval_system.py

![alt text](image-3.png)

![alt text](image-4.png)

![alt text](image-5.png)

可以看到这个recall还是比较低的，这是因为我们刚才没有认真写preprocess_question，刚才就是分词+去掉停用词，刚才调代码是为了跑通Whoosh先。
现在我们来"对问题进行预处理，去除停用词，并选择合适的方法（如词性标注、依存句法以及语义角色标注等）抽取问题中的关键词"

对于Whoosh的Query Formulation，recall比较低的时候，我们使用OR查询是对的，只要有一个关键词匹配就可以了，如果precision低才适合AND查询。

对于算法，Whoosh默认使用的是 BM25F ，看起来已经考虑的够多了这个算法。




In [ ]:
import jieba.posseg as pseg # Import pseg for POS tagging

def preprocess_question(question_text, pos_tags_to_keep=None):
    """
    Preprocesses a question string using POS tagging to extract keywords.
    Args:
        question_text (str): The input question.
        pos_tags_to_keep (set, optional): A set of POS tags to consider for keywords. 
                                         Defaults to nouns, proper nouns, and verbs.
    Returns:
        list: A list of deduplicated keywords.
    """
    if pos_tags_to_keep is None:
        pos_tags_to_keep = {'n', 'nr', 'ns', 'nt', 'nz', 'v'} # Nouns, proper nouns, verbs

    words_with_pos = pseg.lcut(question_text) # List of (word, flag) pairs
    keywords = []
    seen_keywords = set()

    for word, flag in words_with_pos:
        word_clean = word.strip().lower()
        # Check if the word is not a stopword, not just whitespace,
        # and its POS tag (or the beginning of it, e.g., 'nrfg' starts with 'nr') is in our desired set.
        if word_clean and word_clean not in STOPWORDS and not word_clean.isspace():
            is_kept_pos = False
            for kept_tag in pos_tags_to_keep:
                if flag.startswith(kept_tag):
                    is_kept_pos = True
                    break
            
            if is_kept_pos:
                if word_clean not in seen_keywords:
                    keywords.append(word_clean)
                    seen_keywords.add(word_clean)
    return keywords

In [ ]:
我们做了词性分析来提高关键词提取的准确性，确保只提取有意义的词汇，这样BM25F会更好的考虑分数累计、命中数量。

准备评测的时候，我们发现代码评测速度有点慢，刚才我们写的代码里面有些地方值得优化，比如 query parser和search对象不应该反复创建。

![alt text](image-7.png)

这一次我们达到了还算满意的效果，0.8070左右


我在代码中还问了一个示例问题
"2014年南京青年奥林匹克运动会有哪些项目？"
这是从train.json随便找的一个问题。

![alt text](image-8.png)

结果发现系统检索结果为空，如果单独分别检索南京”、“奥林匹克运动会”和“哪些项目”都可以得到合理的答案。

仔细检查代码发现刚才评测到0.8070的代码没错，是我示例的代码忘记传入检索策略为OR了。

In [ ]:
qp = QueryParser("content", schema=index.schema, group=whoosh.qparser.OrGroup)

whoosh.qparser.OrGroup 是关键代码

## 2.构建问题分类器。

Huggingface Tranformers库非常适合做这一个子任务。



In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    pipeline
)


助教建议我们使用  DMetaSoul/sbert-chinese-general-v1 作为预训练模型。

我们可以使用 Hugging Face transformers 库和 Trainer API，结合最新的 Qwen 系列模型（我们将使用一个较小的 Qwen 版本，如 Qwen2-0.5B和Qwen-0.6B）来快速构建、微调和评估问题分类器。

同时 DMetaSoul/sbert-chinese-general-v1 也是很好加载的，接口是一样的。


中途我们遇到了一些报错，比如pad token不存在

https://github.com/huggingface/transformers/blob/main/src/transformers/models/qwen3/modeling_qwen3.py

还比如

最后  DMetaSoul/sbert-chinese-general-v1 分类的效果如下
![alt text](image-10.png)
![alt text](image-9.png)

Fine-grained Accuracy: 0.8450
Coarse-grained Accuracy: 0.9289

而 Qwen3-0.6B 分类的效果如下

![alt text](image-12.png)
![alt text](image-11.png)


Fine-grained Accuracy: 0.9040
Coarse-grained Accuracy: 0.9471

以上训练使用了相同的 batch size，都是16，lr也是一样的。

qwen3的效果明显更好，而且对示例问题的分类更加自信。